In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import glob
from Training.FCN.label import Label 

import os

import numpy as np
import scipy.ndimage as nd
import skimage
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from matplotlib import pyplot as plt
from skimage import measure, segmentation
from skimage.draw import polygon, rectangle
from skimage.transform import resize

import config
import Extraction.mask.maskHelper as mh
from Extraction.clustering import run_clustering
from Training.SNN.model import DataLoaderClustering, Network
from Training.SNN.pageHandler import PageHandler, Pixel
from Training.FCN.unet import UNet
from Extraction.page import Page

from PIL import Image

In [ ]:
output_folder = './time_output/'

m_a = glob.glob(output_folder + '/[0-9]*.jpg.full')
m_b = glob.glob(output_folder + '/[0-9]*.jpg.cluster')

labels = Label('time_label.xml', size=None, resize=0.9)

In [ ]:
def calculate_iou(shape_a, shape_b):
    shape_a = shape_a.astype(int)
    shape_b = shape_b.astype(int)
    intersection = np.sum(np.bitwise_and(shape_a, shape_b))
    union = np.sum(np.bitwise_or(shape_a, shape_b))
    
    return intersection/union

def intersection_over_label_same(shape_a, label):
    shape_a = shape_a.astype(int)
    label = label.astype(int)
    intersection = np.sum(np.bitwise_and(shape_a, label))
    label_size = np.sum(label)
    
    return intersection/label_size

def intersection_over_label(shape_a, label):
    shape_a = shape_a.astype(int)
    label = label.astype(int)
    intersection = np.sum(np.bitwise_and(shape_a, label))
    label_size = np.sum(shape_a)
    
    return intersection/label_size

In [ ]:
labelHandler = labels.annotations
      



In [ ]:
import pickle

def calculate_article_label_overlap(m_a, labelHandler):
    maxes_a = []
    rests_a = []

    best_a = []
    best_b = []
    full = []
    ious = []
    for i in range(0, len(m_a)):
        file = open(m_a[i], 'rb')
        mask = pickle.load(file)
        file.close()

        shape = mask.shape
        #print("new file")
        
        best_labels = []
        article_res = []
        full_labels = []
        for ann in labelHandler[i]['annotations']:
            
            label_mask = np.zeros(mask.shape)
            rr, cc = ann['box']
            label_mask[rr,cc] = 1
            full_labels.append(label_mask)
            max = 0
            total = 0
            max_label = None
            for segment_id in np.unique(mask):
                article = np.zeros(mask.shape)
                article[mask == segment_id] = 1
                iou = intersection_over_label_same(article, label_mask)
                if iou > max:
                    max = iou
                    max_label = segment_id
                    
                total += iou
            maxes_a.append(max)
            best_labels.append(max_label)
            rests_a.append(total-max)

        for seg_id in np.unique(best_labels):
            article = np.zeros(mask.shape)
            article[mask == seg_id] = 1
            max_best = 0
            total_best = 0
            
            max_iou = 0

            best_label2 = None
            other = 0
            for ann in labelHandler[i]['annotations']:
                label_mask = np.zeros(mask.shape)
                rr, cc = ann['box']
                label_mask[rr,cc] = 1
                
                iou = intersection_over_label(article, label_mask)
                same = intersection_over_label_same(article, label_mask)
                c_iou = calculate_iou(article, label_mask)
                if same > max_best:
                    max_best = same
                    max_iou = c_iou
                    other = iou
                    best_label2 = article
                total_best += iou
                
            best_a.append(max_best)
            best_b.append((total_best-other))
            ious.append(max_iou)
            article_res.append({'i': max_best, 'j': (total_best-other), 'label': best_label2})
        #print(len(article_res))
        full.append({'a': m_a[i], 'b': article_res ,'file': labelHandler[i]['file'], 'labels': full_labels})
    return (np.array(best_a), np.array(best_b), full, ious)

In [ ]:
import pickle

def calculate_article_label_overlap_c(m_a, labelHandler):
    maxes_a = []
    rests_a = []

    best_a = []
    best_b = []
    full = []
    ious = []
    for i in range(0, len(m_a)):
        mask = m_a[i]

        shape = mask.shape
        #print("new file")
        
        best_labels = []
        article_res = []
        full_labels = []
        for ann in labelHandler[i]['annotations']:
            
            label_mask = np.zeros(mask.shape)
            rr, cc = ann['box']
            label_mask[rr,cc] = 1
            full_labels.append(label_mask)
            max = 0
            total = 0
            max_label = None
            for segment_id in np.unique(mask):
                article = np.zeros(mask.shape)
                article[mask == segment_id] = 1
                iou = intersection_over_label_same(article, label_mask)
                if iou > max:
                    max = iou
                    max_label = segment_id
                    
                total += iou
            maxes_a.append(max)
           #plt.imshow(max, alpha=0.5)
            #plt.imshow(max_label, alpha=0.5)
            #plt.show()
            best_labels.append(max_label)
            rests_a.append(total-max)

        for seg_id in np.unique(best_labels):
            article = np.zeros(mask.shape)
            article[mask == seg_id] = 1
            max_best = 0
            total_best = 0
            
            max_iou = 0

            best_label2 = None
            other = 0
            for ann in labelHandler[i]['annotations']:
                label_mask = np.zeros(mask.shape)
                rr, cc = ann['box']
                label_mask[rr,cc] = 1
                
                iou = intersection_over_label(article, label_mask)
                same = intersection_over_label_same(article, label_mask)
                c_iou = calculate_iou(article, label_mask)
                if same > max_best:
                    max_best = same
                    max_iou = c_iou
                    other = iou
                    best_label2 = article
                total_best += iou
                
            best_a.append(max_best)
            best_b.append((total_best-other))
            ious.append(max_iou)
            article_res.append({'i': max_best, 'j': (total_best-other), 'label': best_label2})
        #print(len(article_res))
        full.append({'a': m_a[i], 'b': article_res ,'file': labelHandler[i]['file'], 'labels': full_labels})
    return (np.array(best_a), np.array(best_b), full, ious)

In [ ]:
config.IMAGE_PATH = "F:\\Code\\Code\\helper\\dataset\\"

time = {'a': {'m': [], 'l': []}, 'b': {'m': [], 'l': []}, 't': []}

for i in range(0, len(m_a)):
    x = m_a[i].split('\\')[1].split('.')[0]
    
    for j in range(0, len(labelHandler)):
        y = labelHandler[j]['file'].split('.')[0]
        
        if x == y:
            file = m_a[i].split('\\')[1].split('.')[0] + '.jpg'
            img = Image.open(config.IMAGE_PATH + file)
            size = img.size[0]
            time['a']['m'].append(m_a[i])
            time['a']['l'].append(labelHandler[j])
            time['t'].append(len(labelHandler[j]['annotations']))
            time['b']['m'].append(m_b[i])
            time['b']['l'].append(labelHandler[j])

In [ ]:
import pandas as pd
def create_table(c, d, t, add = 0):

    rows = []
    indixes = []
    d = np.clip(d, 0, 1)
    for i in np.arange(0,1.1,0.1):
        row = []
        for j in np.arange(1.1,0,-0.1):
            k = c[d <= j]
            o = (len(k[k >= i]) / len(c))
            
            o = o + add *( 1 - o)
            
            row.append(f"{o:.2f}")
        rows.append(row)
        indixes.append(f"{i:.2f}")
    
    df = pd.DataFrame(rows[:-1],
                    index=pd.Index(indixes[:-1], name='i'),
                    columns=pd.Index(indixes[::-1], name='j'))
    return df

In [ ]:
time_results = calculate_article_label_overlap(time['a']['m'], time['a']['l'])

In [ ]:
df = create_table(time_results[0], time_results[1], time['t'])
df.style.set_caption("Time Magazin").set_table_styles([{
    'selector': 'caption',
    'props': [
        ('font-size', '20px')
    ]
}])


In [ ]:
time['b']['cm'] = []
import math

for i in range(len(time['b']['m'])):
    file = open(time['b']['m'][i], 'rb')
    cluster = pickle.load(file)
    file.close()
    
    dimension = (time['b']['l'][i]['dimension'][1] / 0.9, time['b']['l'][i]['dimension'][0] / 0.9)
    mask = np.zeros((math.ceil(dimension[0]), math.ceil(dimension[1])))
    
    
    offset = (mask.shape - np.multiply(cluster.shape, 50)) / 2
    for y, carr in enumerate(cluster):
        for x, c in enumerate(carr):
            y1 = int(offset[0] + y * 50)
            y2 = int(50 + y1)
            x1 = int(offset[1] + x * 50)
            x2 = int(50 + x1)
            mask[y1:y2,x1:x2] = c
            
    
    time['b']['cm'].append(mask)

In [ ]:
time_results_c = calculate_article_label_overlap_c(time['b']['cm'], time['a']['l'])

In [ ]:
df = create_table(time_results_c[0], time_results_c[1], time['t'])
df.style.set_caption("Time Magazin - Cluster").set_table_styles([{
    'selector': 'caption',
    'props': [
        ('font-size', '20px')
    ]
}])


In [ ]:
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw=None, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (M, N).
    row_labels
        A list or array of length M with the labels for the rows.
    col_labels
        A list or array of length N with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if ax is None:
        ax = plt.gca()

    if cbar_kw is None:
        cbar_kw = {}

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar

    # Show all ticks and label them with the respective list entries.
    ax.set_xticks(np.arange(data.shape[1]), labels=col_labels)
    ax.set_yticks(np.arange(data.shape[0]), labels=row_labels)
    
    ax.xaxis.set_label_position('top') 
    ax.set_xlabel("Overlap with other labels");
    ax.set_ylabel("Overlap with ground truth");

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "white"),
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [ ]:
import matplotlib
df = create_table(time_results_c[0], time_results_c[1], time['t'])
overlap_own = [0, 0.1, 0.2, 0.3, 0.4, 0.5,
              0.6, 0.7, 0.8, 0.9]
overlap_other = [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3,0.2,0.1, 0]



cluster_v = df.values.astype(float)

from matplotlib.pyplot import figure
fig, ax = plt.subplots(figsize=(8,8))
im = heatmap(cluster_v, overlap_own, overlap_other, ax=ax,
                   cmap="YlGn")
texts = annotate_heatmap(im, valfmt="{x:.2f}")

fig.tight_layout()
plt.savefig("time_result.pdf")
plt.show()

In [ ]:
df = create_table(time_results[0], time_results[1], time['t'])

cluster_v = df.values.astype(float)

from matplotlib.pyplot import figure
fig, ax = plt.subplots(figsize=(8,8))
im = heatmap(cluster_v, overlap_own, overlap_other, ax=ax,
                   cmap="YlGn")
texts = annotate_heatmap(im, valfmt="{x:.2f}")

fig.tight_layout()
plt.show()

In [ ]:
bf = calculate_article_label_overlap(bild_der_frau['a']['m'], bild_der_frau['a']['l'])
gb = calculate_article_label_overlap(das_goldene_blatt['a']['m'], das_goldene_blatt['a']['l'])

In [ ]:
bf_cs = calculate_article_label_overlap(bild_der_frau['b']['m'], bild_der_frau['b']['l'])
gb_cs = calculate_article_label_overlap(das_goldene_blatt['b']['m'], das_goldene_blatt['b']['l'])

In [ ]:
plt.scatter(bf[0], bf[1],alpha=0.1)
plt.scatter(gb[0], gb[1],alpha=0.1)
plt.show()

In [ ]:
plt.scatter(bf_cs[0], bf_cs[1],alpha=0.1)
plt.scatter(gb_cs[0], gb_cs[1],alpha=0.1)
plt.show()


In [ ]:
bild_der_frau['t'] = sum(bild_der_frau['t'])
das_goldene_blatt['t'] = sum(das_goldene_blatt['t'])


In [ ]:
df = create_table(bf[0], bf[1], bild_der_frau['t'])
df.style.set_caption("Bild der Frau - Clustering First").set_table_styles([{
    'selector': 'caption',
    'props': [
        ('font-size', '20px')
    ]
}])

In [ ]:
df = create_table(gb[0], gb[1], das_goldene_blatt['t'])
df.style.set_caption("Das Goldene Blatt - Clustering First").set_table_styles([{
    'selector': 'caption',
    'props': [
        ('font-size', '20px')
    ]
}])

In [ ]:
df = create_table(np.concatenate((bf[0], gb[0])), np.concatenate((bf[1], gb[1])), bild_der_frau['t']+ bild_der_frau['t'])
df.style.set_caption("Clustering First").set_table_styles([{
    'selector': 'caption',
    'props': [
        ('font-size', '20px')
    ]
}])

In [ ]:
df = create_table(bf_cs[0], bf_cs[1], bild_der_frau['t'])
df.style.set_caption("Bild der Frau - Clustering Second").set_table_styles([{
    'selector': 'caption',
    'props': [
        ('font-size', '20px')
    ]
}])

In [ ]:
df = create_table(gb_cs[0], gb_cs[1], das_goldene_blatt['t'])
df.style.set_caption("Das Goldene Blatt - Clustering Second").set_table_styles([{
    'selector': 'caption',
    'props': [
        ('font-size', '20px')
    ]
}])

In [ ]:
df = create_table(np.concatenate((bf_cs[0], gb_cs[0])), np.concatenate((bf_cs[1], gb_cs[1])), bild_der_frau['t']+bild_der_frau['t'])
df.style.set_caption("Clustering Second").set_table_styles([{
    'selector': 'caption',
    'props': [
        ('font-size', '20px')
    ]
}])

In [ ]:
def draw_ij_over_image(ref_a, output):
    for page in ref_a:
        image = page['a'].split('\\')[1]
        file = image.split('.')[0] + '.jpg'
        img = Image.open("G:\\Meine Ablage\\Master\\Bilder_small\\" + page['file'])
        plt.imshow(img)
        mask = np.zeros(page['b'][0]['label'].shape)
        
        k = 1
        # draw.text((x, y),"Sample Text",(r,g,b))
        for c in page['b']:
            mask[c['label'] == 1] = k
            k += 1
            center = [ np.average(indices) for indices in np.where(c['label'] == 1) ]
            plt.text(center[1], center[0], "i: " + f"{c['i']:.2f}")
            plt.text(center[1], center[0] + 20, "j: " + f"{c['j']:.2f}")
            
        plt.imshow(mask, cmap='jet', alpha=0.5)
        
        plt.rcParams['figure.dpi'] = 500
        plt.axis('off')
        plt.savefig(output+'/' + file)
        #plt.show()
        plt.clf()
        '''
        mask_label = np.zeros(page['b'][0]['label'].shape)
        
        k = 1
        for l in page['labels']:
            mask_label[l == 1] = k
            k += 1
         
        plt.imshow(img)   
        plt.imshow(mask_label, cmap='jet', alpha=0.5)
        plt.show()'''
        
        

In [ ]:
draw_ij_over_image(bf[2], "out_f5/cf/bf/")

In [ ]:
draw_ij_over_image(gb[2], "out_f5/cf/gb/")

In [ ]:
draw_ij_over_image(bf_cs[2], "out_f5/cs/bf/")
draw_ij_over_image(gb_cs[2], "out_f5/cs/gb/")

In [ ]:
from PIL import Image, ImageChops

def trim(im):
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        return im.crop(bbox)


In [ ]:
images = ["a.jpg","b.jpg", "c.jpg","d.jpg"]
img_objs = []
for img in images:
    file = glob.glob("F:\\Code\\Final\\Extraction\\example_layouts\\"+img)
    print(file[0])
    img_objs.append(trim(Image.open(file[0])))
    
plt.rcParams["figure.dpi"] = 500
fig, ax = plt.subplots(2, 2)


ax[0][0].imshow(img_objs[0])
ax[0][0].axis("off")
ax[0][0].set_title('a)')
ax[0][1].imshow(img_objs[1])
ax[0][1].axis("off")
ax[0][1].set_title('b)')
ax[1][0].imshow(img_objs[2])
ax[1][0].axis("off")
ax[1][0].set_title('c)')
ax[1][1].imshow(img_objs[3])
ax[1][1].axis("off")
ax[1][1].set_title('d)')
fig.tight_layout()
plt.show()

In [ ]:
images = ["198.jpg","076.jpg", "182.jpg","012.jpg"]
img_objs = []
for img in images:
    file = glob.glob("F:/Code/Article Extractor/final_square//"+img)
    print(file)
    img_objs.append(trim(Image.open(file[0])))
    
fig, ax = plt.subplots(2, 2)

plt.rcParams["figure.dpi"] = 500
ax[0][0].imshow(img_objs[0])
ax[0][0].axis("off")
ax[0][0].set_title('a)')
ax[0][1].imshow(img_objs[1])
ax[0][1].axis("off")
ax[0][1].set_title('b)')
ax[1][0].imshow(img_objs[2])
ax[1][0].axis("off")
ax[1][0].set_title('c)')
ax[1][1].imshow(img_objs[3])
ax[1][1].axis("off")
ax[1][1].set_title('d)')
fig.tight_layout()
plt.show()

In [ ]:
images = ["a.jpg","e.jpg", "c.jpg","d.jpg"]
img_objs = []
for img in images:
    file = glob.glob("F:\Code\Article Extractor\example_layouts\\"+img)
    img_objs.append(trim(Image.open(file[0])))
    
fig, ax = plt.subplots(2, 2)


ax[0][0].imshow(img_objs[0])
ax[0][0].axis("off")
ax[0][0].set_title('a)')
ax[0][1].imshow(img_objs[1])
ax[0][1].axis("off")
ax[0][1].set_title('b)')
ax[1][0].imshow(img_objs[2])
ax[1][0].axis("off")
ax[1][0].set_title('c)')
ax[1][1].imshow(img_objs[3])
ax[1][1].axis("off")
ax[1][1].set_title('d)')
fig.tight_layout()
plt.show()

In [ ]:
images = ["1.png","2.png", "3.png","4.png"]
img_objs = []
for img in images:
    file = glob.glob("F:\Code\Article Extractor\cluster_ex\\"+img)
    img_objs.append(Image.open(file[0]))
    
fig, ax = plt.subplots(2, 2)


ax[0][0].imshow(img_objs[0])
ax[0][0].axis("off")
ax[0][0].set_title('a)')
ax[0][1].imshow(img_objs[1])
ax[0][1].axis("off")
ax[0][1].set_title('b)')
ax[1][0].imshow(img_objs[2])
ax[1][0].axis("off")
ax[1][0].set_title('c)')
ax[1][1].imshow(img_objs[3])
ax[1][1].axis("off")
ax[1][1].set_title('d)')
fig.tight_layout()
plt.show()

In [ ]:
images = ["a1.png", "b1.png", "c1.png", "d1.png",
          "a2.png", "b2.png", "c2.png", "d2.png",
          "a3.png", "b3.png", "c3.png", "d3.png"]
img_objs = []
for img in images:
    file = glob.glob("./mask_process/"+img)
    img_objs.append(Image.open(file[0]))
    
fig, ax = plt.subplots(3, 4)


ax[0][0].imshow(img_objs[0])
ax[0][0].axis("off")
ax[0][0].set_title('a)')
ax[0][1].imshow(img_objs[1])
ax[0][1].axis("off")
ax[0][1].set_title('b)')
ax[0][2].imshow(img_objs[2])
ax[0][2].axis("off")
ax[0][2].set_title('c)')
ax[0][3].imshow(img_objs[3])
ax[0][3].axis("off")
ax[0][3].set_title('d)')
ax[1][0].imshow(img_objs[4])
ax[1][0].axis("off")
ax[1][1].imshow(img_objs[5])
ax[1][1].axis("off")
ax[1][2].imshow(img_objs[6])
ax[1][2].axis("off")
ax[1][3].imshow(img_objs[7])
ax[1][3].axis("off")
ax[2][0].imshow(img_objs[8])
ax[2][0].axis("off")
ax[2][1].imshow(img_objs[9])
ax[2][1].axis("off")
ax[2][2].imshow(img_objs[10])
ax[2][2].axis("off")
ax[2][3].imshow(img_objs[11])
ax[2][3].axis("off")
fig.tight_layout()
plt.show()

In [ ]:
'''
ref_a = []
ref_b = []
c, d, ref_a = do_stuff(m_a, labelHandler)
e, f, ref_b = do_stuff(m_b, labelHandler)
c = np.array(c)
d = np.array(d)
e = np.array(e)
f = np.array(f)

plt.scatter(c, d,alpha=0.1)
plt.scatter(e, f,alpha=0.1)
plt.show()
'''

In [ ]:
'''
def get_N_HexCol(N=5):
    HSV_tuples = [(x * 1.0 / N, 0.5, 0.5) for x in range(N)]
    hex_out = []
    for rgb in HSV_tuples:
        rgb = map(lambda x: int(x * 255), colorsys.hsv_to_rgb(*rgb))
        hex_out.append('#%02x%02x%02x' % tuple(rgb))
    return hex_out

from PIL import ImageFont
from PIL import ImageDraw 

for page in ref_a:
    image = page['a'].split('\\')[1]
    file = image.split('.')[0] + '.jpg'
    img = Image.open("G:\\Meine Ablage\\Master\\Bilder_small\\" + page['file'])
    font = ImageFont.truetype("arial.ttf", 16)
    plt.imshow(img)
    mask = np.zeros(page['b'][0]['label'].shape)
    
    k = 1
    # draw.text((x, y),"Sample Text",(r,g,b))
    colors = len(page['b'])
    for c in page['b']:
        rr, cc = np.where(c['label'] == 1)
        mask[c['label'] == 1] = k
        k += 1
        center = [ np.average(indices) for indices in np.where(c['label'] == 1) ]
        plt.text(center[1], center[0], "i: " + f"{c['i']:.2f}")
        plt.text(center[1], center[0] + 20, "j: " + f"{c['j']:.2f}")
        
    plt.imshow(mask, cmap='jet', alpha=0.5)
    
    plt.rcParams['figure.dpi'] = 500
    plt.axis('off')
    plt.savefig('output_label/' + file)
    plt.clf()'''

In [ ]:
'''
for page in ref_b:
    image = page['a'].split('\\')[1]
    file = image.split('.')[0] + '.jpg'
    img = Image.open("G:\\Meine Ablage\\Master\\Bilder_small\\" + page['file'])
    font = ImageFont.truetype("arial.ttf", 16)
    plt.imshow(img)
    mask = np.zeros(page['b'][0]['label'].shape)
    
    k = 1
    # draw.text((x, y),"Sample Text",(r,g,b))
    colors = len(page['b'])
    for c in page['b']:
        rr, cc = np.where(c['label'] == 1)
        mask[c['label'] == 1] = k
        k += 1
        center = [ np.average(indices) for indices in np.where(c['label'] == 1) ]
        plt.text(center[1], center[0], "i: " + f"{c['i']:.2f}")
        plt.text(center[1], center[0] + 20, "j: " + f"{c['j']:.2f}")
        
    plt.imshow(mask, cmap='jet', alpha=0.5)
    
    plt.rcParams['figure.dpi'] = 500
    plt.axis('off')
    plt.savefig('output_label_cs/' + file)
    plt.clf()'''

In [ ]:
index = []
index2 = []
article = []
article2 = []

for i in np.arange(0,1.1,0.1):
    index.append(f"{i:.1f}")
    index2.append(f"{i:.1f}")
    article.append(len(c[c <= i]))
    article2.append(len(e[e <= i]))

plt.scatter(index, article)
plt.scatter(index2, article2)
plt.show()

In [ ]:
create_table(e, f).style

In [ ]:
def create_images(i, j, folder):
    center = [ np.average(indices) for indices in np.where(th1 >= 255) ]

In [ ]:
plt.scatter(c, np.clip(d, 0, 1.1), alpha=0.05)
plt.show()

plt.boxplot(c)
plt.show()

In [ ]:
percsa = []
percsb = []
percsa2 = []
percsb2 = []
for i in range(0,110,10):
    percsa.append(np.percentile(c, 100-i))
    percsb.append(np.percentile(e, 100-i))
    percsa2.append(np.percentile(np.clip(d,0,1), i))
    percsb2.append(np.percentile(np.clip(f,0,1), i))
    

In [ ]:
plt.plot(percsa)
plt.plot(percsb)
plt.xticks(range(0,11, 1), range(0,110, 10))
plt.show()